In [1]:
import pandas as pd

In [2]:
#트레인 데이터 가져오기
df = pd.read_csv("train.csv")

In [3]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
#테스트 데이터 가져오기
df_test = pd.read_csv("test.csv")

In [5]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [6]:
#passengerID값을 데이터로 넣기위해 passengerID group과 멤버, 멤버 숫자로 분류

def change_passengerID(df):
    #passengerID의 group과 멤버는 '_'로 분리
    df["Passenger_Group"] = df["PassengerId"].apply(lambda x: x.split('_')[0])
    df["Passenger_Group_ID"] = df["PassengerId"].apply(lambda x: x.split('_')[1])
    
    #passengerID의 group으로 묶으면 그룹별 멤버가 몇명인지를 카운트
    df_pg_vc = df["Passenger_Group"].value_counts().reset_index()
    df_pg_vc = df_pg_vc.rename(columns={"index":"Passenger_Group","Passenger_Group":"Num_of_Group_meber"})
    
    #기존 데이터에 그룹멤버의 수를 추가하기
    df_result = pd.merge(left=df, right=df_pg_vc, left_on="Passenger_Group", right_on="Passenger_Group")
    return df_result

In [7]:
df_passenger = change_passengerID(df)

In [8]:
#True, False를 1,0으로 수치화하는데, NA값은 imputer에 넣기 위해 남겨두는 함수
#여기에 추가하여 Age값이 object로 되어 있어 int64로 변경

def change_simple_values(df):
    df.CryoSleep = df.CryoSleep.apply(lambda x: 0 if x==False else (1 if x==True else x))
    
    df.VIP = df.VIP.apply(lambda x: 0 if x==False else (1 if x==True else x))
    
    #test는 Transported가 없으서 이 부분을 제외하기 위해서 if 추가
    if('Transported' in df.columns) :
        df.Transported = df.Transported.apply(lambda x: 0 if x==False else (1 if x==True else x))
    
    df.Age = df.Age.astype('Int64')
    return df

In [9]:
df_numericalize = change_simple_values(df_passenger)

In [10]:
#Cabin은 정보가 복합적이어서 이 파라미터를 분리하여 파라미터화하고
#명목형 변수인 'HomePlanet','Destination', 'Cabin'은 onehotencoding으로 변경
#get_dummies함수에서 NA는 0으로 변수화 되는 것을 확인

def change_destination(df):
    
    #Cabin을 나누기 위해서 NA값을 제외하고 '/'로 나눠서 3개의 파라미터 만들기
    df_cabin_splited = pd.DataFrame(index=df[~(df.Cabin.isna())].index, data=list(df[~(df.Cabin.isna())].loc[:,'Cabin'].apply(lambda x: x.split('/')).values), columns=['Cabin_first', 'Cabin_second','Cabin_third'])
    
    #기존 dataframe에 cabin의 3개 파라미터를 합치는 동작
    df_train = pd.merge(left=df, right=df_cabin_splited, left_index=True, right_index=True, how='left')
    
    #명목형 파라미터들을 onehotencoding
    df_train = pd.get_dummies(data=df_train, columns=['HomePlanet','Destination', 'Cabin_first', 'Cabin_third'])
    
    #기존 'PassengerId','Cabin','Name'를 제거하는데
    #추후 name을 고려해서 입력 데이터를 해볼 필요 있음
    df_train_without_Name=df_train.drop(columns=['PassengerId','Cabin','Name'])
    return df_train_without_Name

In [11]:
df_string_handle = change_destination(df_numericalize)

In [12]:
#Passenger_Group, Passenger_Group_ID, Cabin_second가 string으로 되어 있어서 int64로 변경
def change_str_type(df_train_without_Name):
    #첫행에 NA가 없어서, 이 행의 데이터 중 string값이 있는 변수를 확인
    object_sel = df_train_without_Name.iloc[0,:].apply(lambda x: type(x)==type('a')).values
    
    #string변수의 열들의 값을 int64로 변화
    #na는 변경되지 않는 것을 확인(Cabin_second)
    df_train_without_Name.loc[:,object_sel] = df_train_without_Name.loc[:,object_sel].astype('Int64')
    return df_train_without_Name

In [13]:
df_string_handle2 = change_str_type(df_string_handle)

C:\Users\yuvjj\AppData\Local\Temp\ipykernel_18700\2047930959.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_train_without_Name.loc[:,object_sel] = df_train_without_Name.loc[:,object_sel].astype('Int64')


In [14]:
#결측치를 interative imputer를 사용하여 random forest로 머신러닝을 통해서 채우는 방식을 채택
from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import RandomForestRegressor

In [15]:
imp = enable_iterative_imputer.IterativeImputer(estimator=RandomForestRegressor(n_estimators=10))
df_train_imputed = pd.DataFrame(imp.fit_transform(df_string_handle2), columns=df_string_handle2.columns)

C:\Users\yuvjj\Anaconda3\envs\data_science_for_python\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [16]:
#테스트 데이터도 머신러닝에 들어가게 할 수 있게 처리가 필요

df_test_passenger = change_passengerID(df_test)
df_test_numericalize = change_simple_values(df_test_passenger)
df_string_handle = change_destination(df_test_numericalize)
df_string_handle2 = change_str_type(df_string_handle)

C:\Users\yuvjj\AppData\Local\Temp\ipykernel_18700\2047930959.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_train_without_Name.loc[:,object_sel] = df_train_without_Name.loc[:,object_sel].astype('Int64')


In [17]:
#테스트 데이터의 결측치도 핸들

imp = enable_iterative_imputer.IterativeImputer(estimator=RandomForestRegressor(n_estimators=10))
df_test_imputed = pd.DataFrame(imp.fit_transform(df_string_handle2), columns=df_string_handle2.columns)

C:\Users\yuvjj\Anaconda3\envs\data_science_for_python\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [18]:
#classifier는 randomforestclassifier를 사용

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=10).fit(X=df_train_imputed.loc[:,df_train_imputed.columns.drop('Transported')], y=df_train_imputed["Transported"])
df_rf_predict = model.predict(X=df_test_imputed)
df_result = pd.DataFrame(data=df_rf_predict,columns=['Transported'])

In [19]:
#Kaggle submission 파일 생성

df_result.Transported = df_result.apply(lambda x: True if x['Transported']==1 else False, axis=1)
df_submission = pd.read_csv("sample_submission.csv")
df_submit = pd.concat([df_test.PassengerId, df_result], axis=1)
